In [1]:
%load_ext autoreload
%autoreload 2
import ambry
l = ambry.get_library()
b = l.bundle('d04w001') # Geoschemas
sumlevels_p = l.partition('census.gov-acs_geofile-schemas-2009e-sumlevels')

In [2]:
sumlevels = {}
for row in sumlevels_p.stream(as_dict=True):
    sumlevels[row['sumlevel']] = row['description']

In [65]:
from collections import defaultdict, Counter
from geoid import base62_encode

collector = {}
geoids = {}
descriptions = {}

for p in b.partitions:
    #print "=====", p.identity.name
    l = {}
    for i, c in  enumerate(p.table.columns):
        if i > 5 and c.name not in ('name','geoid', 'memi'):
            l[c.name] = [Counter(), 0]
            descriptions[c.name] = c.description
           
    for i, row in enumerate(p.stream(as_dict=True)):
        if i >= 500:
            break
            
        geoid = row['geoid']
        
        for k in l:
            v = row[k]
            
            
            
            if not str(v).strip():
                continue
            
            
            try:
                # The index is not guarantted to be found in the right position; it could be at the start of the
                # geoid, so we keep track of the most common place it is found
                idx = geoid.index(str(v))
                size = len(str(v))
                
                # Kepp tract of the right end position, not the start, since the end pos is independent of the length
                l[k][0][idx+size] += 1
                l[k][1] = max(l[k][1], size)
                
            except ValueError:
                pass
      
    ordered = []
    for k, v in l.items():
        
        most = v[0].most_common(1)
        
        if most:
            size = v[1]
            start = most[0][0] - size

            ordered.append((k, start, size))
            
    ordered = sorted(ordered, key = lambda r: r[1])

    #for e in ordered:
    #    print " ", e, len(base62_encode(10**e[2]))

    geoids[int(p.grain)] = ordered
    
    for e in ordered:
        collector[e[0]]=(e[2],len(base62_encode(10**e[2])) )
        
# Print out the lengths array
out = []
for k, v in collector.items():
    out.append('\'{}\': {},  # {}'.format(k, v[0], descriptions[k]))
    
print '\n'.join(sorted(out))
    
for  sl in sorted(geoids):
    ordered = geoids[sl]
    
    print str(sl)+':', str([ str(e[0]) for e in ordered ])+',', "#", sumlevels[sl]
            
            
            

'aianhh': 4,  # American Indian Area/Alaska Native Area/ Hawaiian Home Land (Census)
'aihhtli': 1,  # American Indian Trust Land/ Hawaiian Home Land Indicator
'aitsce': 3,  # American Indian Tribal Subdivision (Census)
'anrc': 5,  # Alaska Native Regional Corporation (FIPS)
'blkgrp': 1,  # Block Group
'cbsa': 5,  # Metropolitan and Micropolitan Statistical Area
'cdcurr': 2,  # Current Congressional District ***
'cnecta': 3,  # New England City and Town Combined Statistical Area
'concit': 5,  # Consolidated City
'county': 3,  # County of current residence
'cousub': 5,  # County Subdivision (FIPS)
'csa': 3,  # Combined Statistical Area
'division': 1,  # Census Division
'metdiv': 5,  # Metropolitan Statistical Area- Metropolitan Division
'necta': 5,  # New England City and Town Area
'nectadiv': 5,  # New England City and Town Area Division
'place': 5,  # Place (FIPS Code)
'puma5': 4,  # Public Use Microdata Area 5% File
'region': 1,  # Census Region
'sdelm': 5,  # State-School District (E

In [20]:
from geoid import names, segments
names_map = {v:k for k, v in names.items()}

seen = set()

for k, v in segments.items():
    if k in names_map:
        pass
    else:
        
        name =  '_'.join( e for e in v)
        name = name[0].lower() + name[1:]
        
        if name in seen:
            name += str(k)
        
        seen.add(name)
        
        print "'{}': {},".format(name, k)

'state_aianhh': 260,
'necta_nectadiv_state_county_cousub': 358,
'state_sldl': 620,
'state_aianhh_place': 269,
'aianhh_state_county': 270,
'state_cbsa_metdiv': 323,
'state_sldu': 610,
'state_aianhh280': 280,
'state_place_county': 155,
'aianhh_aitsce_state': 290,
'state_aianhh_aihhtli': 283,
'state_cdcurr_aianhh': 550,
'state_concit': 170,
'state_concit_place': 172,
'state_aianhh_aihhtli286': 286,
'cbsa': 310,
'cbsa_state': 311,
'cbsa_state_place': 312,
'cbsa_state_county': 313,
'cbsa_metdiv': 314,
'cbsa_metdiv_state': 315,
'state_cbsa': 320,
'state_cbsa_place': 321,
'state_cbsa_county': 322,
'state_county_cousub_submcd': 67,
'state_cbsa_metdiv_county': 324,
'state_county_cousub_place': 70,
'necta_state_county': 353,
'state_puma5': 795,
'csa': 330,
'csa_state': 331,
'csa_cbsa': 332,
'csa_cbsa_state': 333,
'cnecta': 335,
'state_county_cousub_place_tract': 80,
'cnecta_necta': 337,
'cnecta_necta_state': 338,
'state_csa': 340,
'state_csa_cbsa': 341,
'state_cnecta': 345,
'state_cnecta_necta':

In [10]:
%load_ext autoreload
%autoreload 2
from geoid.acs import AcsGeoid

for p in b.partitions:
           
    for i, row in enumerate(p.stream(as_dict=True)):
        if i >= 500:
            break
            
        geoid = row['geoid']
        
        try:
            AcsGeoid.parse(geoid)
            
        except Exception as e:
            print geoid, e
            raise
        

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
